In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import BeamDynamics as bd
import copy

In [ ]:
from importlib import reload
reload(bd)

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 9]
plotFont = {
    'family' : 'sans-serif',
    'weight' : 'normal',
    'size'   : 12
}
matplotlib.rc('font', **plotFont)
plt.rc('legend', fontsize=10)
defaultColorCycle = plt.rcParams["axes.prop_cycle"]
# %matplotlib notebook

# Title

## Section 1

In [ ]:
sdfFilePath = '../../FCCeeInjectorBeamApp/BeamDistrs/Positrons_200MeV_Yongke/CTSB-N02-F100-E06-S0.5-T5.0_HTSTest_JNov04_SolC_CLICTW-Ztc200-Ri15-Bc0.50.dat.sdf_txt'
beamHtsClic = bd.load_standard_fwf(sdfFilePath)
beamHtsClic.describe()

In [ ]:
plotDefs = [
    {
        'varName1': 'x', 'varName2': 'y',
        'opacityHist': 0.6,
    },
    {
        'varName1': 'x', 'varName2': 'xp',
        'opacityHist': 0.6,
    },
    {
        'varName1': 'y', 'varName2': 'yp',
        'opacityHist': 0.6,
    },
    {
        'varName1': 'z', 'varName2': 'pz',
        'opacityHist': 0.6,
    },
    {
        'varName1': 't', 'varName2': 'Ekin',
        'opacityHist': 0.6,
    },
    {
        'varName1': 't', 'varName2': 'pz',
        'opacityHist': 0.6,
    },
]

In [ ]:
plotDefs[0]['lims1'] = (-30, 30.)   # [mm]
plotDefs[0]['lims2'] = (-30., 30.)   # [mm]
plotDefs[1]['lims1'] = plotDefs[0]['lims1']   # [mm]
plotDefs[1]['lims2'] = (-25., 25.)   # [mrad]
plotDefs[2]['lims1'] = plotDefs[0]['lims2']   # [mm]
plotDefs[2]['lims2'] = (-25., 25.)   # [mrad]
plotDefs[3]['lims1'] = (9999., 10001.)   # [mm]
plotDefs[3]['lims2'] = (0., 400.)   # [MeV/c]
plotDefs[4]['lims1'] = (62.82, 63.10)   # [ns]
plotDefs[4]['lims2'] = plotDefs[3]['lims2']   # [MeV]
plotDefs[5]['lims1'] = plotDefs[4]['lims1']   # [ns]
plotDefs[5]['lims2'] = plotDefs[3]['lims2']   # [MeV/c]

In [ ]:
filterSpecs = {
    'x': plotDefs[1]['lims1'],
    'xp': plotDefs[1]['lims2'],
    'y': plotDefs[2]['lims1'],
    'yp': plotDefs[2]['lims2'],
}
emitn = {}
emitGeom = {}
emitTraceSpace = {}
alphaTwiss = {}
betaTwiss = {}
gammaTwiss = {}
for planeName in ('x', 'y'):
    emitn[planeName] = bd.compute_emittance(
        beamHtsClic, planeName, norm='normalized', filterSpecs=filterSpecs
    )
    emitGeom[planeName] = bd.compute_emittance(
        beamHtsClic, planeName, norm='geometric', filterSpecs=filterSpecs
    )
    emitTraceSpace[planeName] = bd.compute_emittance(
        beamHtsClic, planeName, norm='tracespace', filterSpecs=filterSpecs
    )
    alphaTwiss[planeName], betaTwiss[planeName], gammaTwiss[planeName] = bd.compute_twiss(
        beamHtsClic, planeName, filterSpecs=filterSpecs
    )
    print(
        'emitn_{0:s} = {1:.1f} pi mm mrad, emitGeom_{0:s} = {2:.1f} pi mm mrad, emitTraceSpace_{0:s} = {3:.1f} pi mm mrad.'.format(
            planeName, emitn[planeName], emitGeom[planeName], emitTraceSpace[planeName]
        )
    )
    print(
        'alphaTwiss_{0:s} = {1:.3f}, betaTwiss_{0:s} = {2:.3f} mm, gammaTwiss_{0:s} = {3:.3f} 1/mm.'.format(
            planeName, alphaTwiss[planeName], betaTwiss[planeName], gammaTwiss[planeName]
        )
    )

In [ ]:
selFa = 4.
gaussianPortions = {1: 0.6827, 2: 0.9545, 3: 0.9973, 4: 0.999937, 6: 0.99999998}
ellipseSpecs = {
    'x': {'alphaTwiss': alphaTwiss['x'], 'betaTwiss': betaTwiss['x']},
    'y': {'alphaTwiss': alphaTwiss['y'], 'betaTwiss': betaTwiss['y']}
}
for planes in (['x'], ['x', 'y']):
    for Fa in (1., 2., 3., 4., 6.):
        selEllipseSpecs = {k: v for k, v in ellipseSpecs.items() if k in planes}
        distrWithinFaSigma, portionWithinFaSigma = bd.distr_within_ellipse(
            beamHtsClic, Fa**2.*emitTraceSpace['x'], selEllipseSpecs
        )
        print(
            'Portion within {:.1f} sigma: {:.3f} (vs. {:.3f} for Gaussian).'.format(
                Fa, portionWithinFaSigma, gaussianPortions[Fa]
            )
        )

## Reference values for FODO design

In [ ]:
emitnRef = 10e3   # [pi mm mrad]
FaRef = 4.
pRef = 200   # [MeV/c]
betaGammaRef = bd.p_to_beta(pRef,-11) * bd.p_to_gamma(pRef,-11)
betaGammaRef

## Select portion theoretically transported
In the assumption of a monochromatic beam

In [ ]:
refEllipseSpecs = {
    'x': {'alphaTwiss': 0, 'betaTwiss': betaTwiss['x']},
    'y': {'alphaTwiss': 0, 'betaTwiss': betaTwiss['y']}
}
FaList = (4., 3., 2., 1.)
distrWithinRef = []
for Fa in FaList:
    distrWithinRef.append(bd.distr_within_ellipse(
        beamHtsClic, Fa**2.*emitnRef/betaGammaRef, refEllipseSpecs
    )[0])

## Plot

In [ ]:
ax = bd.plot_distr(
    [beamHtsClic, *distrWithinRef], plotDefs,
    title='HTS AMD, CLIC L-band TW structs., Bz = 0.5 T', legendLabels=[
        'Full distr.',
        *['Within {:.1f} sigma'.format(Fa) for Fa in FaList]
    ]
)
for ind, Fa in enumerate(FaList):
    bd.plot_ellipse(
        ax[1][0,0], Fa**2.*emitnRef/betaGammaRef, semiAxisOrder=2,
        alphaTwiss=0, betaTwiss=betaTwiss['x'], color=ax[1][0,0].get_children()[ind+1].get_edgecolor()
    )
    bd.plot_ellipse(
        ax[2][0,0], Fa**2.*emitnRef/betaGammaRef, semiAxisOrder=2,
        alphaTwiss=0, betaTwiss=betaTwiss['y'], color=ax[2][0,0].get_children()[ind+1].get_edgecolor()
    )

<div class="alert alert-block alert-success">
Some good news.
</div>

<div class="alert alert-block alert-warning">
Some warning.
</div>

<div class="alert alert-block alert-danger">
Some danger.
</div>